In [9]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import time

In [10]:
%run '0_tools.ipynb'

In [11]:
use_updated_word_dict = True

In [12]:
if (use_updated_word_dict): word_dict = list(np.load(f'{train_vect_path}updated_word_dict.npy'))
else: word_dict = list(np.load(f'{train_vect_path}word_dict.npy'))

In [13]:
len(word_dict)

30111

In [16]:
# text = getFileText()
# print(len(text))

76034


In [7]:
clear_old = False

if (clear_old): 
    vect_key = ''
    start_text_index = 0
    text_num=0
    
else: 
    old_np = np.load(f'{train_vect_path}train_vect.npy')
    vect_key='new_'
    start_text_index = old_np.shape[0]
    text_num = start_text_index

finished = False

In [8]:
if (not clear_old):
    new_np = np.load(f'{train_vect_path}{vect_key}train_vect.npy')
    print(f'{old_np.shape} + {new_np.shape}', end = '')
    final_np = np.concatenate((old_np, new_np), axis=0)
    print(f' = {final_np.shape}.')

(76034, 30113) + (0,)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [9]:
if (not clear_old):
    np.save(f'{train_vect_path}train_vect.npy', final_np)
    print(f'Successfully saved {final_np.shape} array.')

Successfully saved (76034, 30113) array.


In [8]:
text = getFileText()

dict_indices = [i for i in range(len(word_dict))]

list_1 = []

total_texts = len(text)

if (start_text_index == 0): print('Starting at beginning of text corpus.')
else: print(f'Starting at article {start_text_index}/{total_texts}.')
    
start_lap = time.time()
for review_rating in text[start_text_index:]:
    
    try:
        
        rr = review_rating.split('\n - \n')
        
        rating = rr[0]
        
        review = rr[1]
        
        review_len = len(review)
        
        if (review_len < 7): continue

        vector = vectorize(review, word_dict)
        
        if (int(rating)<=2): r = -1
        elif (int(rating)==3): r = 0
        elif (int(rating)>=4): r = 1
            
        list_1.append([int(r), review_len] + vector)

        text_num+=1
        if ((text_num-start_text_index)%1000==0):
                        
            end_lap = time.time()
            np_list = np.array(list_1)
            save_start = time.time()
#             np.save(f'{train_vect_path}{vect_key}train_vect.npy', np_list)
            save_end = time.time()
            
            elapsed_time = end_lap - start_lap
            
            print(f'{text_num-start_text_index}/{total_texts-start_text_index} vectors saved', end = ' ')
            print(f'| {str(elapsed_time)[:5]} time units | Save Time {str(save_end-save_start)[:5]} time units.')
            
            start_lap = time.time()
            
        if(text_num == total_texts-1): 
            np_list = np.array(list_1)
            
            np.save(f'{train_vect_path}{vect_key}train_vect.npy', np_list)
            print(f'{text_num-start_text_index}/{total_texts-start_text_index} vectors saved.')
            finished = True
        
    except: continue

if (not finished):
    np_list = np.array(list_1)
    np.save(f'{train_vect_path}{vect_key}train_vect.npy', np_list)
    print(f'{text_num}/{total_texts} vectors saved.')
    finished = True

Starting at article 76034/76036.
76034/76036 vectors saved.


In [ ]:
if (not finished):
    np_list = np.array(list_1)
    np.save(f'{train_vect_path}{vect_key}train_vect.npy', np_list)
    print(f'{text_num+1}/{total_texts} vectors saved.')

### remove words from the dictionary that do not appear much throughout the corpus
ideally misspelled words, words in other languages, etc.

In [5]:
prune=True

In [6]:
if(prune):
    
    more_low_sums = False
    prune_df = pd.DataFrame(np.load(f'{train_vect_path}train_vect.npy')[:40000])

    sums=[1, 1]

    for i in range(2, prune_df.shape[1]):
        sums.append(prune_df[i].sum())
    
    threshold = 0.0005
    low_sums = []
    
    for s in sums: 
        if (s<threshold): low_sums.append(s)
        
    if (more_low_sums):
        
        threshold1 = threshold*2
        threshold2 = threshold/2
        
        low_sums_1 = []
        low_sums_2 = []
        
        for s in sums:
            if (s<threshold1): low_sums_1.append(s)
            if (s<threshold2): low_sums_2.append(s)
            
        print(f'Threshold = {threshold1}: {len(low_sums_1)} values.')
        print(f'Threshold = {threshold}: {len(low_sums)} values.')
        print(f'Threshold = {threshold2}: {len(low_sums_2)} values.')
        
    else: print(f'Threshold = {threshold}: {len(low_sums)} values.')
    
    prune_df_sz = prune_df.shape[0]
    prune_df.loc[prune_df_sz] = sums
    
    columns_remove = (prune_df < threshold).loc[prune_df_sz, 2:]
    columns_remove = pd.concat([pd.Series([False, False]), columns_remove])
    prune_df = prune_df.drop(columns = prune_df.columns[columns_remove])

    
    update_word_dict = set()

    for word_index in (prune_df.columns.tolist()):
        word = word_dict[word_index-2]
        update_word_dict.add(word)

    update_word_dict_np = np.array(list(update_word_dict))
    np.save(f'{train_vect_path}updated_word_dict.npy', update_word_dict_np)
    print(f'Removed words, {update_word_dict_np.shape[0]}/{len(word_dict)} remain.')

Threshold = 0.0005: 37270 values.
Removed words, 30111/67379 remain.
